In [1]:
import cv2
import numpy as np
from PIL import Image
from PIL import ImageTk
import tkinter as tk
import threading
import time
from cv2 import bgsegm

In [2]:
method = 'MOG'
min_contour_area = 3000
frame_box_global = None
frame_thresh_global = None
window = None
image_box = None
image_thresh = None
lock = threading.Lock()

In [3]:
class Background_Subtractor_Processor(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter   
    def run(self):
        camera = cv2.VideoCapture(0)
        global method, min_contour_area
        #Tạo các subtractor (dùng để sau này tạo mask)
        bg_subtractor_MOG = cv2.bgsegm.createBackgroundSubtractorMOG()
        bg_subtractor_MOG2 = cv2.createBackgroundSubtractorMOG2()
        kernel_GMG = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
        bg_subtractor_GMG = cv2.bgsegm.createBackgroundSubtractorGMG()

        while(1):
            _, frame_current = camera.read()
            #Tạo mask 
            if method == 'MOG':
                fgmask = bg_subtractor_MOG2.apply(frame_current)
            elif method == 'MOG2':
                fgmask = bg_subtractor_MOG2.apply(frame_current)
            else:
                fgmask = bg_subtractor_GMG.apply(frame_current)
                fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel_GMG) 

            frame_processed = cv2.bitwise_and(frame_current,frame_current,mask = fgmask) #AND 2 frame lại để lấy được foreground
            frame_processed = cv2.threshold(frame_processed, 20, 255, cv2.THRESH_BINARY)[1] #Apply threshhold = 20
            frame_processed = cv2.cvtColor(frame_processed, cv2.COLOR_BGR2GRAY) #Chuyển sang ảnh xám
#             frame_processed = cv2.medianBlur(frame_processed,9)

            #Tìm các contour
            (_, contours, _) = cv2.findContours(frame_processed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
            for contour in contours: 
                if cv2.contourArea(contour) > min_contour_area: 
                    (x, y, w, h) = cv2.boundingRect(contour) 
                    cv2.rectangle(frame_current, (x, y), (x + w, y + h), (0, 255, 0), 3) #Vẽ box màu xanh lá

#             cv2.imshow(method + ' - Motion',frame_current)
#             cv2.imshow(method,frame_processed)
            
            global frame_box_global, frame_thresh_global, lock
            lock.acquire()
            frame_box_global = frame_current
            frame_thresh_global = frame_processed
            lock.release()

        camera.release()
        cv2.destroyAllWindows()

In [4]:
thread1 = Background_Subtractor_Processor(1, "Thread-1", 1)
thread1.start()
time.sleep(2)

In [5]:
#Các hàm callback cho GUI
def set_contour(area):
    global min_contour_area
    min_contour_area = int(area)
    
def change_method_MOG():
    global method
    method = 'MOG'
    
def change_method_MOG2():
    global method
    method = 'MOG2'
    
def change_method_GMG():
    global method
    method = 'GMG'
    
def change_image_box():
    global frame_box_global
    if frame_box_global is None:
        return
    global window, image_box   
    frame_box_global=cv2.cvtColor(frame_box_global, cv2.COLOR_BGR2RGB) 
#     frame_box_global = cv2.resize(frame_box_global, (352,240))
    photo = ImageTk.PhotoImage(Image.fromarray(frame_box_global))
    image_box.configure(image=photo)
    image_box.image = photo
    image_box.grid(row=0, column=0, columnspan = 3)
    window.after(50, change_image_box)

def change_image_thresh():
    global frame_thresh_global
    if frame_thresh_global is None:
        return
    global window, image_thresh  
#    frame_thresh_global = cv2.resize(frame_thresh_global, (352,240))
    photo = ImageTk.PhotoImage(Image.fromarray(frame_thresh_global))
    image_thresh.configure(image=photo)
    image_thresh.image = photo
    image_thresh.grid(row=0, column=3)
    window.after(50, change_image_thresh)

In [6]:
window = tk.Tk()
window.title('Background Subtractor')

image_box = tk.Label(window)
image_thresh = tk.Label(window)

scale_contour = tk.Scale(window, 
                        orient ='horizontal', 
                        from_ = 0, to = 5000,
                        length = 200,
                        sliderlength = 60,
                        resolution = 50,
                        command = set_contour)
scale_contour.grid(row=2, column=3)
scale_contour.set(min_contour_area)

tk.Label(window,
        text = 'Adjust min contour area').grid(row=3, column=3)

tk.Button(window, 
           text="MOG", 
           command = change_method_MOG).grid(row=2, column=0)

tk.Button(window, 
           text="MOG2", 
           command = change_method_MOG2).grid(row=2, column=1)

tk.Button(window, 
           text="GMG", 
           command = change_method_GMG).grid(row=2, column=2)

In [7]:
window.after(50, change_image_box)
window.after(50, change_image_thresh)
window.mainloop()